In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import json
import sys
import os
from IPython.display import clear_output



In [ ]:

import os
import sys
import warnings
from IPython.display import clear_output


os.environ["TOKENIZERS_PARALLELISM"] = "false"
sys.path.append("..")
warnings.filterwarnings("ignore", category=FutureWarning)

clear_output()


In [ ]:

from google.colab import drive

drive.mount('/content/gdrive/')

my_local_drive='/content/gdrive/MyDrive/Colab Notebooks/mlcq'
sys.path.append(my_local_drive)

%cd $my_local_drive

%pwd

%ls


Mounted at /content/gdrive/
/content/gdrive/MyDrive/Colab Notebooks/mlcq
mlcq_method_mld.json


In [ ]:
# MODEL_NAME = "microsoft/codebert-base"
MODEL_NAME = "answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)


schema_min = {
    "id": []
}
schema_mean = {
    "id": []
}
schema_max = {
    "id": []
}

for i in range(1, 770):
    feature = f"f{i:03}"
    schema_min[feature] = []
    schema_mean[feature] = []
    schema_max[feature] = []



PADDING = "max_length"
TRUNCATION = False

MAX_LENGTH = 8192
STRIDE = MAX_LENGTH - 256

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [ ]:
schema_min["isGodClass"] = []
schema_mean["isGodClass"] = []
schema_max["isGodClass"] = []
schema_min["isDataClass"] = []
schema_mean["isDataClass"] = []
schema_max["isDataClass"] = []


dataset_path = "./mlcq_method_mld.json"

with open(dataset_path, "r") as f:
    dataset = json.load(f)

# dataset = dataset[:10]

print("HELLO")


for index in range(len(dataset)):
    display(index)
    # if index % 100 == 0:
    #     clear_output()

    instance = dataset[index]

    code = instance["source_code"]

    encoding = tokenizer(
        code,
        add_special_tokens = False,
        truncation = TRUNCATION,
        padding = PADDING,
        max_length = MAX_LENGTH
    )
    print("4")

    input_ids = encoding['input_ids']
    print("LOL:", len(input_ids))


    windows = []

    effective_window = MAX_LENGTH - 2  # for CLS and SEP

    for i in range(0, len(input_ids), effective_window - STRIDE):
        print("6")

        window = input_ids[i: i + effective_window]
        print("7")

        # Add special tokens: CLS at beginning, SEP at end.
        window = [tokenizer.cls_token_id] + \
            window + [tokenizer.sep_token_id]
        print("8")

        # Pad if needed to ensure consistent length (max_length)
        if len(window) < MAX_LENGTH:
            window = window + [tokenizer.pad_token_id] * \
                (MAX_LENGTH - len(window))

        print("9")

        windows.append(window)
        print("10")


        if i + effective_window >= len(input_ids):
            break


    print("11")

    embeddings = []

    for window in windows:
        print(len(windows))
        inputs = {
            "input_ids": torch.tensor([window], dtype=torch.long),
            "attention_mask": torch.tensor([[1 if token != tokenizer.pad_token_id else 0 for token in window]], dtype=torch.long)
        }
        print(len(windows))

        with torch.no_grad():
            outputs = model(**inputs)

        print(len(windows))

        embedding = outputs.last_hidden_state[:, 0, :].squeeze()

        print("END")

        embeddings.append(embedding)

    min_embedding = torch.min(torch.stack(embeddings), dim=0)[0]
    mean_embedding = torch.mean(torch.stack(embeddings), dim=0)
    max_embedding = torch.max(torch.stack(embeddings), dim=0)[0]

    schema_min["id"].append(index + 1)
    schema_mean["id"].append(index + 1)
    schema_max["id"].append(index + 1)

    for k in range(1, 769):
        feature = f"f{k:03}"
        schema_min[feature].append(min_embedding[k-1].item())
        schema_mean[feature].append(mean_embedding[k-1].item())
        schema_max[feature].append(max_embedding[k-1].item())

    schema_min["isDataClass"].append(
        1 if "DataClass" in instance["code_smells"] else 0)
    schema_mean["isDataClass"].append(
        1 if "DataClass" in instance["code_smells"] else 0)
    schema_max["isDataClass"].append(
        1 if "DataClass" in instance["code_smells"] else 0)

    schema_min["isGodClass"].append(
        1 if "GodClass" in instance["code_smells"] else 0)
    schema_mean["isGodClass"].append(
        1 if "GodClass" in instance["code_smells"] else 0)
    schema_max["isGodClass"].append(
        1 if "GodClass" in instance["code_smells"] else 0)


HELLO


0

4
LOL: 8192
6
7
8
9
10
6
7
8
9
10
11
2
2
2
END
2
2
2
END


1

4
LOL: 8192
6
7
8
9
10
6
7
8
9
10
11
2
2
2
END
2
2
2
END


2

4
LOL: 8192
6
7
8
9
10
6
7
8
9
10
11
2
2
2
END
2
2
2
END


3

4
LOL: 8192
6
7
8
9
10
6
7
8
9
10
11
2
2


In [ ]:
mlcq_class_min_embedding_csv = "./output/mlcq_class_min_embedding.csv"
mlcq_class_mean_embedding_csv = "./output/mlcq_class_mean_embedding.csv"
mlcq_class_max_embedding_csv = "./output/mlcq_class_max_embedding.csv"

mlcq_class_min_embedding_df = pd.DataFrame(schema_min)
mlcq_class_mean_embedding_df = pd.DataFrame(schema_mean)
mlcq_class_max_embedding_df = pd.DataFrame(schema_max)

mlcq_class_min_embedding_df.to_csv(mlcq_class_min_embedding_csv, index=False)
mlcq_class_mean_embedding_df.to_csv(mlcq_class_mean_embedding_csv, index=False)
mlcq_class_max_embedding_df.to_csv(mlcq_class_max_embedding_csv, index=False)